In [2]:
# https://www.miai.vn/2020/12/22/bert-series-chuong-2-nghich-mot-chut-voi-hugging-face/
import transformers
import torch
import numpy as np
from transformers import BertModel, BertTokenizer

In [3]:
model = BertModel.from_pretrained('bert-base-uncased')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [4]:
#Tokenizer câu văn bản
sent = "I want to learn Python"
print("Câu văn = ", sent)
token = tokenizer.encode(sent)
print("Token array = ", token)

Câu văn =  I want to learn Python
Token array =  [101, 1045, 2215, 2000, 4553, 18750, 102]


In [5]:
#Decode token
d_sent = tokenizer.decode(token)
print("Decode sentence = ", d_sent)

Decode sentence =  [CLS] i want to learn python [SEP]


In [7]:
# Trích xuất đặc trưng câu văn
# Okie, vậy là chúng ta đã token, số hóa được câu văn rồi. Giờ chúng ta sẽ đưa câu văn vào BERT để trích ra đặc trưng nhé:

# Chuyển token array thành tensor
sent_tensor = torch.tensor([token])
print("Sentence tensor = ", sent_tensor)
print("Sentence tensor size = ", sent_tensor.size())
print("Sentence tensor type = ", sent_tensor.dtype)


# Xây dựng attention_mask, ở đây là mảng toàn 1 vì ta quan tâm đến all các token
attention_mask = torch.tensor([np.ones(7)])
print("attention mask type= ", attention_mask.dtype)
print("attention_mask size = ", attention_mask.size())

# Trích đặc trưng
with torch.no_grad():
    last_hidden_states = model(input_ids = sent_tensor, attention_mask = attention_mask)

# In kích thước và giá trị đặc trưng ra màn hình
# print("last hidden states type = ", last_hidden_states.dtype)
print("last hidden states size = ", last_hidden_states[0].size())
print(last_hidden_states[0])

Sentence tensor =  tensor([[  101,  1045,  2215,  2000,  4553, 18750,   102]])
Sentence tensor size =  torch.Size([1, 7])
Sentence tensor type =  torch.int64
attention mask type=  torch.float64
attention_mask size =  torch.Size([1, 7])
last hidden states size =  torch.Size([1, 7, 768])
tensor([[[ 0.2363,  0.2132, -0.0683,  ..., -0.1378,  0.2019,  0.4946],
         [ 0.4133,  0.1613, -0.0224,  ..., -0.0159,  0.4814,  0.4306],
         [ 0.6542,  0.3478,  0.8414,  ...,  0.0991,  0.0711,  0.2553],
         ...,
         [ 0.0472,  0.2704,  0.4221,  ..., -0.0792,  0.3993, -0.4490],
         [-0.2343, -0.2161, -0.4735,  ...,  0.3925,  0.1962, -0.2317],
         [ 0.9054,  0.2575, -0.1419,  ..., -0.0431, -0.8176, -0.3018]]])


# Lưu ý
- Thứ nhất, trong pytorch ra dùng lệnh .size() để in ra kích thước tensor.
- Thứ hai, attention mask là gì? Đó là một ma trận/vector có kích thước bằng với vector token và có giá trị là 0/1 để chỉ ra các token nào Bert cần quan tâm và token nào không cần quan tâm. Trong trường hợp này, chúng ta quan tâm all các token nên để là 1 hết. Khi chúng ta xử lý nhiều câu văn bản có độ dài khác nhau, ta sẽ padding thêm các giá trị đặc biệt, và khi đó ta bảo model là : hãy đừng quan tâm đến các cái tao padding thêm vào bằng cách set các giá trị tương ứng trong attetion mask = 0
- Thứ ba, output embedding mà chúng ta mong muốn sẽ có size là (1,7,768) . Điều này có nghĩa là có tất cả 7 embedding vector cho 7 từ trong câu (bao gồm cả 2 cái từ đặc biệt [CLS] và [SEP] đó) và mỗi vector có độ dài 768.